In [152]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


## Connecting to the cloud EC2

In [180]:
import paramiko

In [181]:
# Set up the SSH client
ssh = paramiko.SSHClient()
ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())

In [182]:
# Connect to the EC2 instance
private_key_path = "C:/Users/sebou/Downloads/test_key.pem"
key = paramiko.RSAKey.from_private_key_file(private_key_path)
ssh.connect(hostname="ec2-18-133-73-36.eu-west-2.compute.amazonaws.com",
            username="ec2-user",
            pkey=key)

In [183]:
my_command = """
u seb;
ls;
"""
# Execute a command (example: list directory contents)
stdin, stdout, stderr = ssh.exec_command(my_command)
print(stdout.read().decode('utf-8'))

_2_sqoop_incremental_import.sh
_3_sqoop_incremental_import.sh
_4_sqoop_incremental_load.sh
data.txt
departments.csv
employees.csv
file1.txt
incremental_load_2.sh
incremental_load_3.sh
incremental_load.sh
mapper.py
output
part-m-00001_copy_1
part-m-00003
project_info_seb.java
QueryResult.java
reducer.py
seb_table.java
sqoop_import.sh
sqoop_incremental_import.sh
test_2.csv
test.csv
the_z_table.java
thursday_table.java
ze_table.java
_z_incremental_load.sh
zipcodes20.csv



## Using Sqoop 
#### To import tables from Postgres to Hive (hdfs)

In [184]:
# Example Sqoop command 
sqoop_command = """
#!/bin/bash

# Variables
HOSTNAME='ec2-3-9-191-104.eu-west-2.compute.amazonaws.com'
DBNAME='testdb'
USERNAME='consultants'
PASSWORD='WelcomeItc@2022'
TABLE='the_z_table'
TARGET_DIR='/tmp/USUK30/seb/hive/the_z_table/'

# Fetch the maximum id value from Hive
LAST_VALUE=$(beeline -u 'jdbc:hive2://ip-172-31-3-80.eu-west-2.compute.internal:10000/default;' --silent=true -e "SELECT MAX(id) FROM the_z_table;" | grep -o '[0-9]*' | tail -n 1)

echo "Last recorded ID: $LAST_VALUE"
echo "Starting new import from ID greater than $LAST_VALUE"

# Perform the incremental Sqoop import
sqoop import \
    --connect jdbc:postgresql://${HOSTNAME}:5432/${DBNAME} \
    --username ${USERNAME} \
    --password ${PASSWORD} \
    --table ${TABLE} \
    --incremental append \
    --check-column id \
    --last-value ${LAST_VALUE} \
    --target-dir ${TARGET_DIR} \
    --m 1 \
    --as-textfile
"""


In [163]:
# Execute the Sqoop command
stdin, stdout, stderr = ssh.exec_command(sqoop_command) 

In [164]:
%time
# Check the command's output
output = stdout.read().decode('utf-8') 
print("STDOUT:", output) 

STDOUT: Last recorded ID: 56
Starting new import from ID greater than 56



In [165]:
# Check for errors
errors = stderr.read().decode('utf-8')
if errors:
    print("STDERR:", errors) 


STDERR: SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/opt/cloudera/parcels/CDH-7.1.7-1.cdh7.1.7.p0.15945976/jars/log4j-slf4j-impl-2.13.3.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/opt/cloudera/parcels/CDH-7.1.7-1.cdh7.1.7.p0.15945976/jars/slf4j-log4j12-1.7.30.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]
SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/opt/cloudera/parcels/CDH-7.1.7-1.cdh7.1.7.p0.15945976/jars/log4j-slf4j-impl-2.13.3.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/opt/cloudera/parcels/CDH-7.1.7-1.cdh7.1.7.p0.15945976/jars/slf4j-log4j12-1.7.30.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanatio

### We go to Hive 

In [166]:
from pyhive import hive
import pandas as pd

- **We set a tunnel**
go to the command prompt, then execute the following (adapt if needed)
```
ssh -i "C:/Users/sebou/Downloads/test_key.pem" -L 10000:ip-172-31-3-80.eu-west-2.compute.internal:10000 ec2-user@ec2-18-133-73-36.eu-west-2.compute.amazonaws.com -N
```

In [167]:
# after tunnel 
from pyhive import hive
# Establish connection to Hive through the SSH tunnel
conn = hive.Connection(host='localhost', port=10000, username='hive')


In [168]:
import warnings

# Suppress specific UserWarnings related to pandas read_sql
warnings.filterwarnings('ignore', message='pandas only supports SQLAlchemy connectable')


In [169]:
# Try establishing the connection
try:
    print("Connected to Hive!")
    # Example query
    query = "SELECT * FROM the_z_table LIMIT 10"
    df = pd.read_sql(query, conn)
    print(df)
except Exception as e:
    print(f"Error connecting to Hive: {e}")


Connected to Hive!
   the_z_table.id the_z_table.name  the_z_table.age
0               1         John Doe               30
1               2       Jane Smith               32
2               3     Alex Johnson               25
3               4     Maria Garcia               28
4               5         Chen Wei               33
5               6     Olivia Brown               29
6               7      Liam Miller               31
7               8      Emma Wilson               27
8               9       Noah Davis               34
9              10       Ava Taylor               26


In [170]:
query = "SELECT * FROM the_z_table "

In [179]:
df = pd.read_sql(query, conn)
df.head()

DatabaseError: Execution failed on sql: SELECT * FROM the_z_table 
unexpected exception
unable to rollback

,the_z_table.id,the_z_table.name,the_z_table.age
0,1,John Doe,30
1,2,Jane Smith,32
2,3,Alex Johnson,25
3,4,Maria Garcia,28
4,5,Chen Wei,33


In [142]:
df.tail()

,the_z_table.id,the_z_table.name,the_z_table.age
41,52,aMarion Lembert,34
42,53,aAlexia Hanson,25
43,54,aMariam Ahnia,28
44,55,aChon Wong,33
45,56,aOliver Town,29


In [177]:
df.tail()

,the_z_table.id,the_z_table.name,the_z_table.age
45,56,aOliver Town,29
46,57,bTeny Zern,23
47,58,bLarz Pison,32
48,59,bNiu Gip,28
49,60,bNius Gips,28


In [ ]:
# Remember to close the connection when done
# ssh.close()

In [176]:
# Close the connection
ssh.close()

In [3]:
# !pip install pyhive
# if needed

In [9]:
#!pip install thrift
# if needed

In [15]:
#!pip install pyhive


In [17]:
#!pip install thrift_sasl


In [6]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [18]:
from pyhive import hive
import subprocess
import os


In [30]:
# Variables
hostname = 'ec2-3-9-191-104.eu-west-2.compute.amazonaws.com'
dbname = 'testdb'
username = 'consultants'
# Assume PASSWORD is set as an environment variable for security reasons
# password = os.environ.get("DB_PASSWORD")
password ='WelcomeItc@2022'
table = 'the_z_table'
target_dir = '/tmp/USUK30/seb/hive/the_z_table/'


run on the command prompt :
```
ssh -i "C:\Users\sebou\Downloads\test_key.pem" -L 10000:ip-172-31-3-80.eu-west-2.compute.internal:10000 ec2-user@ec2-18-133-73-36.eu-west-2.compute.amazonaws.com -N
```

In [35]:
conn = hive.Connection(host='localhost', port=10000, username='hive')


In [40]:
from pyhive import hive
# Assuming the connection is already established as conn
cursor = conn.cursor()
cursor.execute("SELECT MAX(id) FROM the_z_table")
last_recorded_id = cursor.fetchone()[0]
print(f"Last recorded ID: {last_recorded_id}")


Last recorded ID: 51


In [41]:
import subprocess

command = 'ssh -i "C:/Users/sebou/Downloads/test_key.pem" ec2-user@your-remote-host "sqoop import --connect jdbc:postgresql://ec2-3-9-191-104.eu-west-2.compute.amazonaws.com:5432/testdb --username consultants --password WelcomeItc@2022 --table the_z_table --m 1 --target-dir /tmp/USUK30/seb/hive/the_z_table --as-textfile"'
subprocess.run(command, shell=True)


CompletedProcess(args='ssh -i "C:/Users/sebou/Downloads/test_key.pem" ec2-user@your-remote-host "sqoop import --connect jdbc:postgresql://ec2-3-9-191-104.eu-west-2.compute.amazonaws.com:5432/testdb --username consultants --password WelcomeItc@2022 --table the_z_table --m 1 --target-dir /tmp/USUK30/seb/hive/the_z_table --as-textfile"', returncode=255)

In [43]:
command = 'ssh -i "C:\\Users\\sebou\\Downloads\\test_key.pem" ec2-user@ec2-18-133-73-36.eu-west-2.compute.amazonaws.com "sqoop import --connect jdbc:postgresql://ec2-3-9-191-104.eu-west-2.compute.amazonaws.com:5432/testdb --username consultants --password WelcomeItc@2022 --table the_z_table --m 1 --target-dir /tmp/USUK30/seb/hive/the_z_table --as-textfile"'
subprocess.run(command, shell=True)


CompletedProcess(args='ssh -i "C:\\Users\\sebou\\Downloads\\test_key.pem" ec2-user@ec2-18-133-73-36.eu-west-2.compute.amazonaws.com "sqoop import --connect jdbc:postgresql://ec2-3-9-191-104.eu-west-2.compute.amazonaws.com:5432/testdb --username consultants --password WelcomeItc@2022 --table the_z_table --m 1 --target-dir /tmp/USUK30/seb/hive/the_z_table --as-textfile"', returncode=1)

In [37]:
import subprocess

# Prepare the Sqoop command
sqoop_cmd = [
    "sqoop", "import",
    "--connect", f"jdbc:postgresql://{hostname}:5432/{dbname}",
    "--username", username,
    "--password", password,
    "--table", table,
    "--incremental", "append",
    "--check-column", "id",
    "--last-value", str(last_recorded_id),
    "--target-dir", target_dir,
    "--m", "1",
    "--as-textfile"
]

# Execute the Sqoop command
subprocess.run(sqoop_cmd, check=True)


FileNotFoundError: [WinError 2] The system cannot find the file specified

In [32]:
# Fetch the last recorded ID from Hive
cursor = conn.cursor()
cursor.execute(f"SELECT MAX(id) FROM {table}")
last_value = cursor.fetchone()[0]
print(f"Last recorded ID: {last_value}")


Last recorded ID: 51


In [33]:
# Incremental Sqoop import command
sqoop_command = f"""
sqoop import \
    --connect jdbc:postgresql://{hostname}:5432/{dbname} \
    --username {username} \
    --password {password} \
    --table {table} \
    --incremental append \
    --check-column id \
    --last-value {last_value} \
    --target-dir {target_dir} \
    --m 1 \
    --as-textfile
"""


In [34]:
# Execute Sqoop command
process = subprocess.run(sqoop_command, shell=True, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
print(process.stdout.decode())
